make it into a city and find the fastest route based on variables

* Road types - highway, street, residential, bridges...
* speed limits
* traffic density - low med high
* accidents / closed roads
* weather
* time of day - time sensitive routes
* bus/train/ emergency vehicles??
* traffic lights?





* save map before and after with path
* save and name multiple maps


In [4]:
import pygame, os 
import json
import math
from timeit import default_timer as timer
from queue import PriorityQueue
WIDTH = 600
HEIGHT = 600
win = pygame.display.set_mode((WIDTH, HEIGHT))
clock =pygame.time.Clock()
pygame.display.set_caption('Thingy')

RED = (255,0,0)
GREEN = (0,255,0)
Blue = (0,0,255)
YELLOW = (255,255,0)
WHITE = (255,255,255)
BLACK = (0,0,0)
PURPLE = (128,0,128)
ORANGE = (255,165,0)
GREY = (128,128,128)
TURQOUISE = (64,224,208)


BACKGROUND = (19,103,11) #grass green ish
PATH_CLOSE = (135,101,9) 
PATH_OPEN = (194,176,17)
GRID= (20,0,0)
WALL= (36,26,1)
START= GREEN
END= RED
PATH= TURQOUISE

HIGHWAY = (50,50,255) #dark blue
STREET = (170,170,170) #medium grey
RESIDENTIAL = (200,200,200) #Light grey
ACCIDENT= (138,6,6) #dark red
SPEED_LIMIT25= (255,255,153)#Light yellow
SPEED_LIMIT35= (255,204,102)#light orange
SPEED_LIMIT65= (153,255,153)#mint
MORNING= (255,223,186) #peach
AFTERNOON= (255,255,204) #pale yellow
NIGHT= (153,204,255) #sky blue
BUS= (225,225,0) #brigth yellow
TRAIN= (160,0,160) #purple
TRAFFIC_LIGHTS= (0,255,0) #green

x=1

HIGHWAY_COST = 0.5*x
STREET_COST = 1*x
RESIDENTIAL_COST= 2*x
ACCIDENT_COST= 5*x
SPEED_LIMIT25_COST= .65*x
SPEED_LIMIT35_COST= .35*x
SPEED_LIMIT65_COST= .25*x
MORNING_COST= 1.0
AFTERNOON_COST= 1.2
NIGHT_COST= 0.8
BUS_COST= 1.5
TRAIN_COST= 4.0
TRAFFIC_LIGHTS_COST= 0.2




class Node:
    def __init__ (self, row, col, width, total_rows):
        self.row=row
        self.col=col
        self.width=width
        self.x=row*width
        self.y=col*width
        self.color=BACKGROUND
        self.total_rows=total_rows
        self.speed_cost=1
        self.time_modifier=1
        self.delay=0

        self.node_costs = {
            'HIGHWAY': HIGHWAY_COST,
            'STREET': STREET_COST,
            'RESIDENTIAL': RESIDENTIAL_COST,
            'ACCIDENT': ACCIDENT_COST,
            'SPEED_LIMIT25': SPEED_LIMIT25_COST,
            'SPEED_LIMIT35': SPEED_LIMIT35_COST,
            'SPEED_LIMIT65': SPEED_LIMIT65_COST,
            'MORNING': MORNING_COST,
            'AFTERNOON': AFTERNOON_COST,
            'NIGHT': NIGHT_COST,
            'BUS': BUS_COST,
            'TRAIN': TRAIN_COST,
            'TRAFFIC_LIGHTS': TRAFFIC_LIGHTS_COST
        }

    def get_cost(self):
        # Depending on the current color, return the corresponding cost
        if self.is_highway():
            return self.node_costs['HIGHWAY']
        elif self.is_street():
            return self.node_costs['STREET']
        elif self.is_residential():
            return self.node_costs['RESIDENTIAL']
        elif self.is_accident():
            return self.node_costs['ACCIDENT']
        elif self.is_speed_limit25():
            return self.node_costs['SPEED_LIMIT25']
        elif self.is_speed_limit35():
            return self.node_costs['SPEED_LIMIT35']
        elif self.is_speed_limit65():
            return self.node_costs['SPEED_LIMIT65']
        elif self.is_morning():
            return self.node_costs['MORNING']
        elif self.is_afternoon():
            return self.node_costs['AFTERNOON']
        elif self.is_night():
            return self.node_costs['NIGHT']
        elif self.is_bus():
            return self.node_costs['BUS']
        elif self.is_train():
            return self.node_costs['TRAIN']
        elif self.is_traffic_lights():
            return self.node_costs['TRAFFIC_LIGHTS']
        else:
            return self.speed_cost  # Default cost if no specific state is found
        
    def get_pos(self):
        return self.row,self.col
    
    def reset(self):
        self.color=BACKGROUND
        self.speed_cost=1
        self.time_modifier=1
        self.delay=0

    def make_close(self):
        self.color = PATH_CLOSE

    def make_open(self):
        self.color = PATH_OPEN

    def make_wall(self):
        self.color = WALL
    
    def make_start(self):
        self.color = START

    def make_end(self):
        self.color = END

    def make_path(self):
        self.color = PATH

    def is_highway(self):
        return self.color == HIGHWAY

    def is_street(self):
        return self.color == STREET

    def is_residential(self):
        return self.color == RESIDENTIAL

    def is_accident(self):
        return self.color == ACCIDENT

    def is_speed_limit25(self):
        return self.color == SPEED_LIMIT25

    def is_speed_limit35(self):
        return self.color == SPEED_LIMIT35

    def is_speed_limit65(self):
        return self.color == SPEED_LIMIT65

    def is_morning(self):
        return self.color == MORNING

    def is_afternoon(self):
        return self.color == AFTERNOON

    def is_night(self):
        return self.color == NIGHT

    def is_bus(self):
        return self.color == BUS

    def is_train(self):
        return self.color == TRAIN

    def is_traffic_lights(self):
        return self.color == TRAFFIC_LIGHTS

    def is_closed(self):
        return self.color==PATH_CLOSE
    
    def is_open(self):
        return self.color==PATH_OPEN
    
    def is_wall(self):
        return self.color==WALL
    
    def is_start(self):
        return self.color==START
    
    def is_end(self):
        return self.color==END
    
    def is_background(self):
        return self.color==BACKGROUND
    
    def draw(self,win):
        pygame.draw.rect(win, self.color, (self.x,self.y,self.width,self.width))

    def update_neighbors(self, grid):
        self.neighbors=[]
        #checking -y axis
        if self.row<self.total_rows-1 and not grid[self.row+1][self.col].is_wall():
            self.neighbors.append(grid[self.row+1][self.col])
        #checking y axis
        if self.row>0 and not grid[self.row-1][self.col].is_wall():
            self.neighbors.append(grid[self.row-1][self.col])
        #checking -x axis
        if self.col<self.total_rows-1 and not grid[self.row][self.col+1].is_wall():
            self.neighbors.append(grid[self.row][self.col+1])
        #checking x axis
        if self.col>0 and not grid[self.row][self.col-1].is_wall():
            self.neighbors.append(grid[self.row][self.col-1])

    def __lt__ (self,other):
        return False
    
def h(p1,p2):
    x1,y1=p1
    x2,y2=p2
    return abs(x1-x2)+abs(y1-y2)
    
#remakes the shortest path from start to finish
def reconstruct_path(node_path, current, draw, counter_start):
    pygame.display.set_caption('thingy2 (making poop tube...)')
    path_count=0
    total_cost=0
    while current in node_path:
        current = node_path[current]
        current.make_path()
        path_count+=1
        total_cost+=current.get_cost()
        draw()
    counter_end=timer()
    time_elapsed = counter_end-counter_start
    pygame.display.set_caption(f"time wasted: {format(time_elapsed, '.2f')}s | Cells touched:  {len(node_path) +1} | shortest tube: {path_count+1} Tubes | total cost {total_cost}")
    
#algorithm to search for a path to the end
def algorithm(draw, grid, start, end, counter_start):
    count=0
    open_set=PriorityQueue()
    open_set.put((0,count,start))
    node_path={}
    g_score={spot: float('inf') for row in grid for spot in row}
    g_score[start]=0

    f_score = {spot: float('inf') for row in grid for spot in row}
    f_score[start]=h(start.get_pos(), end.get_pos())

    open_set_hash = {start}

    original_colors={}
    modified_spots=[]

    for row in grid:
        for spot in row:
            if not spot.is_wall() and not spot.is_start() and not spot.is_end():
                original_colors[spot] = spot.color

    while not open_set.empty():
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit()
        current = open_set.get()[2]
        open_set_hash.remove(current)

        if current == end:
            reconstruct_path(node_path, end, draw, counter_start)
            end.make_end()
            return True
            
        for neighbor in current.neighbors:
            temp_g_score=g_score[current]+neighbor.get_cost()

            if temp_g_score < g_score[neighbor]:
                node_path[neighbor]=current
                g_score[neighbor]=temp_g_score
                f_score[neighbor] = temp_g_score
                if neighbor not in open_set_hash:
                    count+=1
                    open_set.put((f_score[neighbor], count, neighbor))
                    open_set_hash.add(neighbor)
                    if not neighbor.is_wall() and not neighbor.is_start() and not neighbor.is_end() and not neighbor.is_background():
                        if not (neighbor.is_highway() or neighbor.is_street() or neighbor.is_residential() or
                                neighbor.is_accident() or neighbor.is_speed_limit25() or
                                neighbor.is_speed_limit35() or neighbor.is_speed_limit65() or
                                neighbor.is_morning() or neighbor.is_afternoon() or neighbor.is_night() or
                                neighbor.is_bus() or neighbor.is_train() or neighbor.is_traffic_lights()):
                            neighbor.color = PATH_OPEN
                            modified_spots.append(neighbor)
        if current != start and current != end:
            if current.color != PATH_OPEN:  # Don't overwrite already open nodes
                current.color = PATH_CLOSE 
                modified_spots.append(current)
        draw()
        if current!= start:
            current.make_close()
    for spot in modified_spots:
        if spot.color!=PATH and spot.color!=PATH_OPEN and spot.color!=PATH_CLOSE:
            spot.color=original_colors.get(spot, BACKGROUND)
    pygame.display.set_caption('thingy can not find poop tube')

def make_grid(rows, width):
    grid=[]
    gap=width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Node(i, j, gap, rows)
            grid[i].append(spot)
    return grid
    

def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GRID, (0,i*gap), (width,i*gap))
        for j in range(rows):
            pygame.draw.line(win, GRID, (i*gap,0), (i*gap,width))

#setting up the drwaing function for the walls
def draw_grid_wall(rows, grid):
    for i in range(rows):
        for j in range(rows):
            if i==0 or i==rows-1 or j==0 or j==rows-1:
                spot=grid[i][j]
                spot.make_wall()

#drawing the stuff
def draw(win, grid, rows, width):
    for row in grid:
        for spot in row:
            spot.draw(win)
    draw_grid(win, rows, width)
    draw_grid_wall(rows,grid)
    pygame.display.update()

def get_mouse_pos(pos,rows,width):
    gap=width//rows
    y,x=pos
    row=y // gap
    col=x // gap
    return row,col
def main_func_yo(win, width):
    GIRTH= 30
    grid = make_grid(GIRTH, width)
    Start = None
    End = None
    Run = True
    current_type='wall'

    while Run:
        draw(win, grid, GIRTH, width)
        for e in pygame.event.get():
            if e.type==pygame.QUIT:
                Run = False
            #regular click
            if pygame.mouse.get_pressed()[0]:
                pos = pygame.mouse.get_pos()
                row,col=get_mouse_pos(pos, GIRTH, width)
                spot=grid[row][col]
                if not Start and spot != End:
                    Start = spot
                    Start.make_start()
                    
                elif not End and spot !=Start:
                    End=spot
                    End.make_end()
                
                elif spot != Start and spot != End:
                    if current_type == 'wall':
                        spot.make_wall()
                    elif current_type == 'highway':
                        spot.color = HIGHWAY
                    elif current_type == 'street':
                        spot.color = STREET
                    elif current_type == 'residential':
                        spot.color = RESIDENTIAL
                    elif current_type == 'accident':
                        spot.color = ACCIDENT
                    elif current_type == 'speed_limit25':
                        spot.color = SPEED_LIMIT25
                    elif current_type == 'speed_limit35':
                        spot.color = SPEED_LIMIT35
                    elif current_type == 'speed_limit65':
                        spot.color = SPEED_LIMIT65
                    elif current_type == 'morning':
                        spot.color = MORNING
                    elif current_type == 'afternoon':
                        spot.color = AFTERNOON
                    elif current_type == 'night':
                        spot.color = NIGHT
                    elif current_type == 'bus':
                        spot.color = BUS
                    elif current_type == 'train':
                        spot.color = TRAIN
                    elif current_type == 'traffic_lights':
                        spot.color = TRAFFIC_LIGHTS
            #right click
            if pygame.mouse.get_pressed()[2]:
                pos=pygame.mouse.get_pos()
                row,col=get_mouse_pos(pos, GIRTH, width)
                spot=grid[row][col]
                if spot==Start:
                    Start=None
                if spot ==End:
                    End=None
                spot.reset()

            if e.type ==pygame.KEYDOWN:
                if not Start and not End:
                    #setting start and end
                    pygame.display.set_caption('thingy (put the first and last thingy down yo)')
                if e.key == pygame.K_SPACE and Start and End:
                    counter_start=timer()
                    #search
                    pygame.display.set_caption('thingy (thingying)')
                    for row in grid:
                        for spot in row:
                            spot.update_neighbors(grid)
                    
                    algorithm(lambda: draw(win, grid, GIRTH, width), grid, Start, End, counter_start)

                if e.key == pygame.K_r:
                    Start=None
                    End=None
                    pygame.display.set_caption('thingy (uses stuff to do things)')
                    grid = make_grid(GIRTH, width)


                if e.key == pygame.K_9:
                    # Save the current grid state
                    save_grid(grid)
                
                #if e.key == pygame.K_0:
                    # Load the saved grid state
                    #grid = load_grid(filename="grid_state.json", rows=GIRTH, width=width)


                if e.key == pygame.K_h:
                    current_type = 'highway'
                    pygame.display.set_caption('Current terrain: HIGHWAY')
                elif e.key == pygame.K_s:
                    current_type = 'street'
                    pygame.display.set_caption('Current terrain: STREET')
                elif e.key == pygame.K_d:
                    current_type = 'residential'
                    pygame.display.set_caption('Current terrain: RESIDENTIAL')
                elif e.key == pygame.K_a:
                    current_type = 'accident'
                    pygame.display.set_caption('Current terrain: ACCIDENT')
                elif e.key == pygame.K_2:
                    current_type = 'speed_limit25'
                    pygame.display.set_caption('Current terrain: SPEED LIMIT 25')
                elif e.key == pygame.K_3:
                    current_type = 'speed_limit35'
                    pygame.display.set_caption('Current terrain: SPEED LIMIT 35')
                elif e.key == pygame.K_6:
                    current_type = 'speed_limit65'
                    pygame.display.set_caption('Current terrain: SPEED LIMIT 65')
                elif e.key == pygame.K_m:
                    #current_type = 'morning'
                    x=MORNING_COST
                    pygame.display.set_caption('Current terrain: MORNING')
                elif e.key == pygame.K_f:
                    #current_type = 'afternoon'
                    x=AFTERNOON_COST
                    pygame.display.set_caption('Current terrain: AFTERNOON')
                elif e.key == pygame.K_n:
                    #current_type = 'night'
                    x=NIGHT_COST
                    pygame.display.set_caption('Current terrain: NIGHT')
                elif e.key == pygame.K_b:
                    current_type = 'bus'
                    pygame.display.set_caption('Current terrain: BUS')
                elif e.key == pygame.K_t:
                    current_type = 'train'
                    pygame.display.set_caption('Current terrain: TRAIN')
                elif e.key == pygame.K_l:
                    current_type = 'traffic_lights'
                    pygame.display.set_caption('Current terrain: TRAFFIC LIGHTS')
                elif e.key == pygame.K_w:
                    current_type = 'wall'
                    pygame.display.set_caption('Current terrain: WALL')
                
    pygame.quit()

# Function to serialize the grid
def save_grid(grid, filename="grid_state.json"):
    grid_state = []
    for row in grid:
        row_state = []
        for spot in row:
            # Save the color of each node (you can extend this to include other properties if necessary)
            node_state = {
                "row": spot.row,
                "col": spot.col,
                "color": spot.color,
                "terrain": spot.color  # We store the terrain type here as well
            }
            row_state.append(node_state)
        grid_state.append(row_state)

    with open(filename, 'w') as file:
        json.dump(grid_state, file, default=str)  # Saving the grid as JSON
    print(f"Grid state saved to {filename}")
    


main_func_yo(win,WIDTH)



In [ ]:
import pygame, os 
import math
from timeit import default_timer as timer
from queue import PriorityQueue
WIDTH = 600
HEIGHT = 600
win = pygame.display.set_mode((WIDTH, HEIGHT))
clock =pygame.time.Clock()
pygame.display.set_caption('Thingy')
RED = (255,0,0)
GREEN = (0,255,0)
Blue = (0,0,255)
YELLOW = (255,255,0)
WHITE = (255,255,255)
BLACK = (0,0,0)
PURPLE = (128,0,128)
ORANGE = (255,165,0)
GREY = (128,128,128)
TURQOUISE = (64,224,208)


BACKGROUND = (19,103,11) #grass green ish
PATH_CLOSE = (135,101,9) 
PATH_OPEN = (194,176,17)
GRID= (20,0,0)
WALL= (36,26,1)
START= GREEN
END= RED
PATH= TURQOUISE

HIGHWAY = (50,50,255) #dark blue
STREET = (170,170,170) #medium grey
RESIDENTIAL = (200,200,200) #Light grey
ACCIDENT= (138,6,6) #dark red
SPEED_LIMIT25= (255,255,153)#Light yellow
SPEED_LIMIT35= (255,204,102)#light orange
SPEED_LIMIT65= (153,255,153)#mint
MORNING= (255,223,186) #peach
AFTERNOON= (255,255,204) #pale yellow
NIGHT= (153,204,255) #sky blue
BUS= (225,225,0) #brigth yellow
TRAIN= (160,0,160) #purple
TRAFFIC_LIGHTS= (0,255,0) #green

HIGHWAY_COST = 0.5
STREET_COST = 1
RESIDENTIAL_COST= 10
ACCIDENT_COST= 5
SPEED_LIMIT25_COST= .65
SPEED_LIMIT35_COST= .35
SPEED_LIMIT65_COST= .25
MORNING_COST= 0.8
AFTERNOON_COST= 1.0
NIGHT_COST= 0.6
BUS_COST= 1.5
TRAIN_COST= 4.0
TRAFFIC_LIGHTS_COST= 0.2




class Node:
    def __init__ (self, row, col, width, total_rows):
        self.row=row
        self.col=col
        self.width=width
        self.x=row*width
        self.y=col*width
        self.color=BACKGROUND
        self.total_rows=total_rows
        self.speed_cost=1
        self.time_modifier=1
        self.delay=0

        self.node_costs = {
            'HIGHWAY': HIGHWAY_COST,
            'STREET': STREET_COST,
            'RESIDENTIAL': RESIDENTIAL_COST,
            'ACCIDENT': ACCIDENT_COST,
            'SPEED_LIMIT25': SPEED_LIMIT25_COST,
            'SPEED_LIMIT35': SPEED_LIMIT35_COST,
            'SPEED_LIMIT65': SPEED_LIMIT65_COST,
            'MORNING': MORNING_COST,
            'AFTERNOON': AFTERNOON_COST,
            'NIGHT': NIGHT_COST,
            'BUS': BUS_COST,
            'TRAIN': TRAIN_COST,
            'TRAFFIC_LIGHTS': TRAFFIC_LIGHTS_COST
        }

    def get_cost(self):
        # Depending on the current color, return the corresponding cost
        if self.is_highway():
            return self.node_costs['HIGHWAY']
        elif self.is_street():
            return self.node_costs['STREET']
        elif self.is_residential():
            return self.node_costs['RESIDENTIAL']
        elif self.is_accident():
            return self.node_costs['ACCIDENT']
        elif self.is_speed_limit25():
            return self.node_costs['SPEED_LIMIT25']
        elif self.is_speed_limit35():
            return self.node_costs['SPEED_LIMIT35']
        elif self.is_speed_limit65():
            return self.node_costs['SPEED_LIMIT65']
        elif self.is_morning():
            return self.node_costs['MORNING']
        elif self.is_afternoon():
            return self.node_costs['AFTERNOON']
        elif self.is_night():
            return self.node_costs['NIGHT']
        elif self.is_bus():
            return self.node_costs['BUS']
        elif self.is_train():
            return self.node_costs['TRAIN']
        elif self.is_traffic_lights():
            return self.node_costs['TRAFFIC_LIGHTS']
        else:
            return self.speed_cost  # Default cost if no specific state is found
        
    def get_pos(self):
        return self.row,self.col
    
    def reset(self):
        self.color=BACKGROUND
        self.speed_cost=1
        self.time_modifier=1
        self.delay=0

    def make_close(self):
        self.color = PATH_CLOSE

    def make_open(self):
        self.color = PATH_OPEN

    def make_wall(self):
        self.color = WALL
    
    def make_start(self):
        self.color = START

    def make_end(self):
        self.color = END

    def make_path(self):
        self.color = PATH

    def is_highway(self):
        return self.color == HIGHWAY

    def is_street(self):
        return self.color == STREET

    def is_residential(self):
        return self.color == RESIDENTIAL

    def is_accident(self):
        return self.color == ACCIDENT

    def is_speed_limit25(self):
        return self.color == SPEED_LIMIT25

    def is_speed_limit35(self):
        return self.color == SPEED_LIMIT35

    def is_speed_limit65(self):
        return self.color == SPEED_LIMIT65

    def is_morning(self):
        return self.color == MORNING

    def is_afternoon(self):
        return self.color == AFTERNOON

    def is_night(self):
        return self.color == NIGHT

    def is_bus(self):
        return self.color == BUS

    def is_train(self):
        return self.color == TRAIN

    def is_traffic_lights(self):
        return self.color == TRAFFIC_LIGHTS

    def is_closed(self):
        return self.color==PATH_CLOSE
    
    def is_open(self):
        return self.color==PATH_OPEN
    
    def is_wall(self):
        return self.color==WALL
    
    def is_start(self):
        return self.color==START
    
    def is_end(self):
        return self.color==END
    
    def is_background(self):
        return self.color==BACKGROUND
    
    def draw(self,win):
        pygame.draw.rect(win, self.color, (self.x,self.y,self.width,self.width))

    def update_neighbors(self, grid):
        self.neighbors=[]
        #checking -y axis
        if self.row<self.total_rows-1 and not grid[self.row+1][self.col].is_wall():
            self.neighbors.append(grid[self.row+1][self.col])
        #checking y axis
        if self.row>0 and not grid[self.row-1][self.col].is_wall():
            self.neighbors.append(grid[self.row-1][self.col])
        #checking -x axis
        if self.col<self.total_rows-1 and not grid[self.row][self.col+1].is_wall():
            self.neighbors.append(grid[self.row][self.col+1])
        #checking x axis
        if self.col>0 and not grid[self.row][self.col-1].is_wall():
            self.neighbors.append(grid[self.row][self.col-1])

    def __lt__ (self,other):
        return False
    
def make_grid(rows, width):
    grid=[]
    gap=width // rows
    for i in range(rows):
        grid.append([])
        for j in range(rows):
            spot = Node(i, j, gap, rows)
            grid[i].append(spot)
    return grid
    

def draw_grid(win, rows, width):
    gap = width // rows
    for i in range(rows):
        pygame.draw.line(win, GRID, (0,i*gap), (width,i*gap))
        for j in range(rows):
            pygame.draw.line(win, GRID, (i*gap,0), (i*gap,width))

#setting up the drwaing function for the walls
def draw_grid_wall(rows, grid):
    for i in range(rows):
        for j in range(rows):
            if i==0 or i==rows-1 or j==0 or j==rows-1:
                spot=grid[i][j]
                spot.make_wall()

#drawing the stuff
def draw(win, grid, rows, width):
    for row in grid:
        for spot in row:
            spot.draw(win)
    draw_grid(win, rows, width)
    draw_grid_wall(rows,grid)
    pygame.display.update()

def saved_graph(win, width):
    GIRTH= 30
    grid = make_grid(GIRTH, width)
    Run = True

    while Run:
        draw(win, grid, GIRTH, width)
        for e in pygame.event.get():
            if e.type==pygame.QUIT:
                Run = False
saved_graph(win,WIDTH)
                    


pygame 2.6.1 (SDL 2.28.4, Python 3.12.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 

In [ ]:


player_pos = pygame.Vector2(screen.get_width() / 2, screen.get_height() / 2)

while running:
    # poll for events
    # pygame.QUIT event means the user clicked X to close your window
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # fill the screen with a color to wipe away anything from last frame
    screen.fill("purple")

    pygame.draw.circle(screen, "red", player_pos, 40)

    keys = pygame.key.get_pressed()
    if keys[pygame.K_w]:
        player_pos.y -= 300 * dt
    if keys[pygame.K_s]:
        player_pos.y += 300 * dt
    if keys[pygame.K_a]:
        player_pos.x -= 300 * dt
    if keys[pygame.K_d]:
        player_pos.x += 300 * dt

    # flip() the display to put your work on screen
    pygame.display.flip()

    # limits FPS to 60
    # dt is delta time in seconds since last frame, used for framerate-
    # independent physics.
    dt = clock.tick(60) / 1000

pygame.quit()